In [1]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from matplotlib.image import imread
import pandas as pd
from PIL import Image
import cv2

In [2]:
import zipfile
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
import warnings

In [3]:
zip_path = 'FracAtlas.zip'
extract_dir = 'fracatlas_extracted'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

images_dir = os.path.join(extract_dir, 'FracAtlas/images')
annotations_dir = os.path.join(extract_dir, 'FracAtlas/Annotations/YOLO')

In [4]:
# Checked to see if each image has a corresponding label

categories = ['Fractured', 'Non_fractured']

for cat in categories:
    category_path = os.path.join(images_dir, cat)

    for img_file in os.listdir(category_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_name = os.path.splitext(img_file)[0]
            annotation_file = img_name + '.txt'
            annotation_path = os.path.join(annotations_dir, annotation_file)

            # if os.path.exists(annotation_path):
            #     print(f"Found annotation for {img_file}: {annotation_path}")
            # else:
            #     print(f"No annotation found for {img_file} at {annotation_path}.")

In [5]:
# Organize the data into {"inputs": images, "labels": labels} structure!

images_dir = 'fracatlas_extracted/FracAtlas/images'
annotations_dir = 'fracatlas_extracted/FracAtlas/Annotations/YOLO'

images = []
labels = []

for category in os.listdir(images_dir):
    category_path = os.path.join(images_dir, category)

    if not os.path.isdir(category_path):
        continue

    for img_file in os.listdir(category_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_name = os.path.splitext(img_file)[0]
            annotation_file = img_name + '.txt'
            annotation_path = os.path.join(annotations_dir, annotation_file)

            if os.path.exists(annotation_path):
                img_path = os.path.join(category_path, img_file)
                images.append(img_path)
                labels.append(category)
            else:
                print(f"No annotation found for {img_file} at {annotation_path}.")

data_dict_old = {
    "inputs": images,
    "labels": labels
}

print(f"{len(images)} image paths and {len(labels)} labels in organized data")

4083 image paths and 4083 labels in organized data


In [6]:
# Do not include corrupted/damaged images

def preprocess(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = img_array / 255.0 # normalize pixel values
        return img_array
    except (OSError, IOError) as e:
        warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
        return None

images = []
labels = []

for img_path, label in zip(data_dict_old["inputs"], data_dict_old["labels"]):
    img = preprocess(img_path)
    if img is not None:
        images.append(img)
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

data_dict = {
    "inputs": images,
    "labels": labels
}

print(f"{len(images)}/{len(data_dict_old['inputs'])} images processed")

<ipython-input-6-f39cfcd6ffb1>:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004073.jpg. Error: image file is truncated (3 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
<ipython-input-6-f39cfcd6ffb1>:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004036.jpg. Error: image file is truncated (14 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
<ipython-input-6-f39cfcd6ffb1>:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004121.jpg. Error: image file is truncated (40 bytes not processed)
  warnings.warn(f"Skipping corrupted image: {img_path}. Error: {e}")
<ipython-input-6-f39cfcd6ffb1>:10: UserWarning: Skipping corrupted image: fracatlas_extracted/FracAtlas/images/Non_fractured/IMG0004258.jpg. Error: image file is truncated (0 bytes not processed)
  warni

4024/4083 images processed


In [7]:
# Encode the output labels to be 0 or 1

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data_dict["labels"])
data_dict["encoded_labels"] = encoded_labels

In [8]:
# Function to convert image to grayscale
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Function to normalize pixel values to [0, 1]
def normalize(image):
    return image / 255.0

# Function to standardize the image data, mean=0 and std=1
def standardize(image):
    mean = np.mean(image)
    std = np.std(image)
    standardized_image = (image - mean) / std
    return standardized_image

In [13]:
# Convert images to grayscale and save
def convert_and_save_grayscale_images(folder_path, save_folder, target_size=(224, 224)):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)  # Make sure save folder exists

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for i, filename in enumerate(image_files):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Skip over corrupt images
        if image is None:
            warnings.warn(f"Skipping corrupt or unreadable image: {image_path}")
            continue

        image_resized = cv2.resize(image, target_size)

        # Convert to grayscale
        grayscale_image = convert_to_grayscale(image_resized)

        # Save the grayscale image
        processed_image_path = os.path.join(save_folder, f"grayscale_{i}.npy")
        np.save(processed_image_path, grayscale_image)

        if i % 10 == 0:
            print(f"Processed and saved {i+1}/{len(image_files)} grayscale images.")

        # Free up memory
        del image, grayscale_image

    print("Grayscale conversion completed.")

# Normalize the grayscale images
def normalize_and_save_images(folder_path, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.npy')]

    for i, filename in enumerate(image_files):
        image_path = os.path.join(folder_path, filename)
        grayscale_image = np.load(image_path)

        # Normalize the grayscale image
        normalized_image = normalize(grayscale_image)

        # Save the normalized image
        processed_image_path = os.path.join(save_folder, f"normalized_{i}.npy")
        np.save(processed_image_path, normalized_image)

        if i % 10 == 0:
            print(f"Processed and saved {i+1}/{len(image_files)} normalized images.")

        # Free up memory
        del grayscale_image, normalized_image

    print("Normalization completed.")

# Standardize the normalized images
def standardize_and_save_images(folder_path, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.npy')]

    for i, filename in enumerate(image_files):
        image_path = os.path.join(folder_path, filename)
        normalized_image = np.load(image_path)

        # Standardize the normalized image
        standardized_image = standardize(normalized_image)

        # Save the standardized image
        processed_image_path = os.path.join(save_folder, f"standardized_{i}.npy")
        np.save(processed_image_path, standardized_image)

        if i % 10 == 0:
            print(f"Processed and saved {i+1}/{len(image_files)} standardized images.")

        # Free up memory
        del normalized_image, standardized_image

    print("Standardization completed.")

In [14]:
# Folder paths
fractured_path = '/content/fracatlas_extracted/FracAtlas/images/Fractured'
non_fractured_path = '/content/fracatlas_extracted/FracAtlas/images/Non_fractured'

# Folders to save the processed images for fractured and non-fractured
grayscale_fractured_folder = '/content/Processed/Grayscale/Processed_Fractured_Grayscale'
grayscale_non_fractured_folder = '/content/Processed/Grayscale/Processed_Non_Fractured_Grayscale'

normalized_fractured_folder = '/content/Processed/Normalize/Processed_Fractured_Normalized'
normalized_non_fractured_folder = '/content/Processed/Normalize/Processed_Non_Fractured_Normalized'

standardized_fractured_folder = '/content/Processed/Final/Processed_Fractured_Final'
standardized_non_fractured_folder = '/content/Processed/Final/Processed_Non_Fractured_Final'

In [15]:
# Convert all fractured and non-fractured images to grayscale and save
convert_and_save_grayscale_images(fractured_path, grayscale_fractured_folder)
convert_and_save_grayscale_images(non_fractured_path, grayscale_non_fractured_folder)

# Normalize grayscale images for both fractured and non-fractured categories
normalize_and_save_images(grayscale_fractured_folder, normalized_fractured_folder)
normalize_and_save_images(grayscale_non_fractured_folder, normalized_non_fractured_folder)

# Standardize normalized images for both fractured and non-fractured categories
standardize_and_save_images(normalized_fractured_folder, standardized_fractured_folder)
standardize_and_save_images(normalized_non_fractured_folder, standardized_non_fractured_folder)

print("All preprocessing steps completed for both fractured and non-fractured images.")

Processed and saved 1/717 grayscale images.
Processed and saved 11/717 grayscale images.
Processed and saved 21/717 grayscale images.
Processed and saved 31/717 grayscale images.
Processed and saved 41/717 grayscale images.
Processed and saved 51/717 grayscale images.
Processed and saved 61/717 grayscale images.
Processed and saved 71/717 grayscale images.
Processed and saved 81/717 grayscale images.
Processed and saved 91/717 grayscale images.
Processed and saved 101/717 grayscale images.
Processed and saved 111/717 grayscale images.
Processed and saved 121/717 grayscale images.
Processed and saved 131/717 grayscale images.
Processed and saved 141/717 grayscale images.
Processed and saved 151/717 grayscale images.
Processed and saved 161/717 grayscale images.
Processed and saved 171/717 grayscale images.
Processed and saved 181/717 grayscale images.
Processed and saved 191/717 grayscale images.
Processed and saved 201/717 grayscale images.
Processed and saved 211/717 grayscale images.